# Machine Translation and Document Search

## Overview

In this notebook, we learn to implement machine translation to translate the English word "hello" to the French word "bonjour". We also learn to implement document search. For example, given a document such as the sentence, "Can I get a refund?", we can search for similar documents such as, "what's your return policy?" Or "may I get my money back?" We learn any sets of skills that we can apply to both of these useful NLP tasks.

# Transforming word vectors

Let's get an overview of machine translation by starting with an example of English to French translation. In order to translate an English word to a French word, one way would be to generate an extensive list of English words and their associated French word. If we ask the human to do this, we would find someone who knows both languages to start making a list. If we want a machine to learn how to do this, we would calculate word embeddings associated with English and word embeddings associated with French. Next, retrieve the English word embedding of a particular English word such as "cat", then find some way to transform the English word embedding into word embedding that has a meaning in the French word vector space. Next, we take the transformed word vector and search for word vectors in the French word vector space that are most similar to it. The most similar words are candidates words for the translation. If the machine does a good job, it's may find the word "chat", which is the French word for "cat". 

To transform vectors using matrices, we can write a code where we define the matrix $R$, then define the vector $x$, multiply $x$ by $R$ using dot product, and the result is another two dimensional vector. 

In [1]:
import numpy as np

R = np.array([[2,  0],
              [0, -2]])
x = np.array([[1, 1]])

np.dot(x, R)

array([[ 2, -2]])

Now that we know that there can be a matrix that transforms our English word vectors into relevant French word vectors, how do we define this transformation matrix, which we'll denote as $R$? We can start with a randomly selected matrix $R$ and then see how it performs when we try to translate the English vectors in matrix $x$ and compare that to the actual French word vectors, which is in the matrix $Y$. 

$$
XR \approx Y
$$

In order for this to work, we will first need to get a subset of English words and different equivalence, get the respective word vectors, and stack the word vectors in their respective matrices $X$ and $Y$. The key here is to keep the rows lined up or to align the word vectors. This means that if the first row of matrix $X$ contains the word "cat", then the first row of the matrix "Y" should contain the French word for "cat", which is "chat".

$$
\left[ 
\begin{matrix}
  [\text{"cat" vector}] \\
  [ \ldots vector] \\
  [\text{"zebra" vector}]\\
\end{matrix}
\right ]\hspace{5cm}  \left [ \begin{matrix}
  [\text{"chat" vector}] \\
  [ \ldots vector] \\
  [\text{"zébresse" vector}] 
\end{matrix} \right ]
$$

We should train a model instead of just save the information of both matrices in a key value mapping because we can just collect a subset of these words to find the transformation matrix. If it works well, then the model can be used to translate words that are not part of the original training set. So, we only need to train on a subset of the English-French vocabulary and not the entire vocabulary. 

To find a good matrix $R$, we compare the translation $X$ times $R$ with the actual French word embeddings in $Y$. We do this by taking the $X$ matrix times the $R$ matrix and subtracting the $Y$ matrix. 

$$
Loss = || XR - Y ||_F
$$

If we start with a random matrix $R$, we can gradually improve this matrix $R$ in a loop, where we compute the gradient by taking the derivative of this loss function with respect to the matrix $R$. 

$$
g = \frac{d}{dR}Loss
$$

Next, we update the matrix $R$ by subtracting the gradient rated by the learning rate $\alpha$. 

$$
R = R - \alpha g
$$

We can either pick a fixed number of times to go through the loop or check the loss at each iteration and break out of the loop when the loss falls between a certain threshold. 

The notation with the double vertical lines measures the **magnitude or the norm of a matrix**. Let's see an example of calculating this norm and then see the general formula. Let's say that the results of $XR - Y$ is a matrix. We pretend for this example that there's only two words in this dictionary, which is the number of rows in the matrix and the word embeddings have two dimensions. So that's the number of columns in the matrix. So matrices $X$, $R$, $Y$ and $A$ are all 2 by 2 matrices. If the matrix $A$ looks like the one below, 

$$
A = \left ( \begin{matrix}
2 & 2 \\
2 & 2 \\
\end{matrix} \right )
$$

Then to calculate its norm, we calculate as

$$
||A||_F = \sqrt{\vphantom{\sum}2^2 + 2^2 + 2^2 + 2^2} = 4
$$

Below is the actual formula. We just take all the elements in the matrix, square them, and add them up. This norm has the subscript F because this is called the **Frobenius norm**. 

$$
||A||_F = \sqrt{\sum\limits_{i=1}^{m}\sum\limits_{j=1}^{n}|a_{ij}|^2}
$$

Let's calculate the Frobenius norm in code. We start with a matrix $A$ and square all the elements, then sum them and extract the square root to get 4. 

In [3]:
A = np.array([[2, 2],
              [2, 2]])
A_squared = np.square(A)
A_Frobenious = np.sqrt(np.sum(A_squared))
print(A_Frobenious)

4.0


In practice, it is easier to minimize the **square of the Frobenius norm**. 

$$
Loss = || XR - Y ||_F^2
$$

In other words, we can cancel out the square root by taking the square. If we go back to our example with matrix $A$, the square root of Frobenius norm cancel it out by squaring the sum. So the square of the Frobenius norm is 16.

$$
||A||_F^2 \equiv \left (\sqrt{\vphantom{\sum}2^2 + 2^2 + 2^2 + 2^2}\right )^2 = 16
$$

Let's go into detail on how to calculate the gradient of the loss function. The loss is defined as the square of the Frobenius norm. 

$$
Loss = || XR - Y ||_F^2
$$

The gradient is the derivative of the loss with respect to the matrix $R$. 

$$
g = \frac{d}{dR}
$$

If it looks like the equation below, the scalar $m$ is the number of rows or words in the subset that we are using for training. 

$$
Loss = \frac{2}{m} \left (\vphantom{\frac{}{}}X^T(XR - Y)\right )
$$

It's easier to take the derivative of this expression rather than dealing with the square root that's in the Frobenius norm. 

## K-nearest neighbors

One key operation needed to find a matching word in the lesson was finding the K-nearest neighbors of a vector. Notice that it transformed word vector after the transformation of its embedding through an $R$ matrix would be in the French word vector space. But it is not going to be necessarily identical to any of the word vectors in the French word vector space. We need to search through the actual French word vectors to find a French word that is similar to the one that we created from the transformation. We may find words such as "salut" or "bonjour", which we can return as the French translation of the word "hello". 

To understand how to find similar word vectors, let's look at how do we find friends who are living nearby? Let's pretend that we are visiting San Francisco in the United States and we are visiting a dear friend Andrew. We also want to visit other friends over the weekend, preferably those who live nearby. One way to do this is to go through the address book and for each friend get their address, calculate how far they are from San Francisco. 

<img src="images/nearest_neighbors.svg" width="70%"/>

So one friend is in Shanghai, the other friend is in Bangalore, and another friend is in Los Angeles. We can sort friends by their distances to San Francisco, then rank them by how close they are. Notice that if we have a lot of friends, this is a very time intensive process.

Notice that two of these friends live in another continent, while the third friend lives in the United States. It is important to note that we may not have been necessary to go through all of friends in the address in order to find the ones closest to us. We might could filter on which friends were all in a general region, such as North America, then we could just search within that sub group of friends. If there is a way to slice up the geographic space into regions (as the dashed lines in the image above), we could search just within those regions. 

When we think about organizing subsets of a dataset efficiently, we may think about placing data into buckets, *i.e.*, hash tables. **Hash tables** are useful tools for any kind of work involving data.

## Hash tables and hash functions

Let us say we have several data items and we want to group them into buckets by some kind of similarity. One bucket can hold more than one item and each item is always assigned to the same bucket. So the results would be the red rectangles and up in bucket number zero, the blue triangles end up in bucket number one, and the green ellipses are assigned to buckets two. 

<img src="images/hashes.svg" width="30%"/>

Now let's think about how we would like to do this with word vectors. First, let's assume that the word vectors have just 1 dimension instead the 300 dimensions. So each word is represented by a single number such as `100`, `14`, `17`, `10` and `97`. We need to find a way to give each vector a hash value which is a key that tells it which bucket it's assigned to. A function that assigns a hash value is called a **hash function**. 

$$
\text{Hash function(vector)}\ \ \ \ \ \ \rightarrow\ \ \ \ \ \ \text{Hash value}
$$

In the example below, there is a hash table which is a set of buckets. In this case, the hash table has ten buckets. Notice how the word vectors 100 and 10 are assigned to bucket 0. The word vector 14 is assigned to buckets 4 and the word vectors 17 and 97 are assigned to bucket 7. 

<img src="images/hash_buckets.svg" width="40%"/>

The formula is the hash function that's being used to assign the word vectors to their respective buckets. The modulo operator (%) takes the remainder after dividing by ten. The remainder is the hash value that tells us where the word vector should be stored. For example, `14` divided by `10` has a remainder of `4`, so it goes to buckets `4`. 

A basic hash table in code would be described as:

In [5]:
def basic_hash_table(value_1, n_buckets):
    def hash_function(value_1, n_buckets):
        return int(value) % n_buckets
    hash_table = {i:[] for i in range(n_buckets)}
    for value in value_1:
        hash_value = hash_function(value, n_buckets)
        hash_table[hash_value].append(value)
    return hash_table

value_1 = [100, 14, 17, 10, 97]
n_buckets = 10
hash_t = basic_hash_table(value_1, n_buckets)

for b in hash_t:
    print('Bucket: {} -> {}'.format(b, hash_t[b]))

Bucket: 0 -> [100, 10]
Bucket: 1 -> []
Bucket: 2 -> []
Bucket: 3 -> []
Bucket: 4 -> [14]
Bucket: 5 -> []
Bucket: 6 -> []
Bucket: 7 -> [17, 97]
Bucket: 8 -> []
Bucket: 9 -> []


Another look at this basic hash table would re-organize values where similar word vectors are into the same bucket.

<img src="images/hash_buckets_fixed.svg" width="40%"/>

This is called **locality sensitive hashing**. Locality is another word for location, sensitive is another word for caring. So locality sensitive hashing is a hashing method that's cares very deeply about assigning items based on where they're located in vector space.

## Locality sensitive hashing

One key method that we use to reduce the computational cost of finding k-nearest neighbors in high-dimensional spaces is **locality-sensitive hashing**. To start thinking about locality-sensitive hashing, let's first assume that we are using word vectors with just two dimensions. So let's say we want to find a way to know that the blue dots are somehow close to each other, and that the gray dots are also related to each other. First, divide the space using the dashed lines, which we call planes. Notice how the blue plane slices up the space into vectors that are above it or below it. The blue vectors all happen to be on the same side of the blue plane. Similarly, the gray vectors happen to be above the gray plane. 

<img src="images/ls_hashing.svg" width="35%"/>

It looks like the planes can help us bucket the vectors into subsets based on their location. A hashing function that is sensitive to the location of the items that it's assigning into buckets. A **plane** would be the magenta line into two-dimensional space, and it actually represents all the possible vectors that would be sitting on that plane. In other words, they would be parallel to the plane, such as the blue vector or the orange vector. We can define a plane with a single vector. The magenta vector is perpendicular to the plane, and it's called the normal vector to that plane. The normal vector is perpendicular to any vectors that lie on the plane. It might help to think about this in three dimensions. Find a sheet of paper and find a pencil. Place the paper on the table and draw some vectors on it, then hold the pencil vertically over the paper. Any vectors on the paper are perpendicular to the pencil. 

<img src="images/vectors_in_space.svg" width="60%"/>

In two dimensions, we are able to see visually when the vector is on one side of the plane or the other. For example, in the image below, we have three sample vectors in blue, orange, and green. 

<img src="images/vectors_dot_product.svg" width="30%"/>

The normal vector to the plane is labeled `P`. Let's focus on vector 1. What if we take the dot product of P with vector one? We get three. Now, let's look at the vector 2. If we take the dot product of P with vector 2, we get zero. Finally, if we take the dot product of P with vector 3, we get negative three. So the dot products are

$$
PV_1^T = 3 \\
PV_2^T = 0 \\
PV_3^T = -3 \\
$$

When the dot product is positive, the vector is on one side of the plane. If the dot product is negative, the vector is on the opposite side of the plane. If the dot product is zero, the vector is on the plane. So what's the dot product doing? To visualize the dot product, imagine one of the vectors such as `P`, as if it's the surface of the Earth. Gravity pulls all objects straight down towards the surface of the Earth. Next, pretend we're standing at the end of the vector, V1. We tie a string to a rock and let gravity pull the rock to the surface of vector `P`. The string is perpendicular to vector `P`. Now, if we draw a vector that's in the same direction of `P` but ends up at the rock, we'll have what's called the projection of vector `V1` onto vector `P`. The magnitude or length of that vector is equal to the dot product of `V1` and `P`. 

<img src="images/vectors_signal.svg" width="80%"/>

Furthermore, if we had this other green vector and projected it onto vector `P`, the projected vector would be pointing in the parallel but opposite direction of `P`. The dot product would be a negative number. This means that the sign of the dot product indicates the direction of the projection with respect to the purple normal vector. So whether the dot product is positive or negative can tell we whether the vector `V1` or `V2` are on one side of the plane or the other. 

Let's use code to check which side of the plane the vector is on. The function side of plane takes in the normal vector `P`, and the vector `v`. We take the dot products and the sign to get a plus one if the dot product is positive, minus one if the dot product is negative, or zero if the dot product is zero. Finally, we can get the vector if it can be represented as a single scalar.

In [9]:
def side_of_plane(P, v):
    dotproduct = np.dot(P, v.T)
    sign_of_dot_product = np.sign(dotproduct)
    sign_of_dot_product_scalar = np.asscalar(sign_of_dot_product)
    return sign_of_dot_product_scalar

P = np.array([1, 1])
V1 = np.array([1, 2])
V2 = np.array([-1, 1])
V3 = np.array([-2, -1])

for i, v in enumerate([V1, V2, V3], start=1):
    scalar = side_of_plane(P, v)
    print('Vector V_{}: {}'.format(i, scalar))

Vector V_1: 1
Vector V_2: 0
Vector V_3: -1


/home/roger/Installed/anaconda3/envs/dl-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  after removing the cwd from sys.path.


## Multiple Planes

Now, we learn about combining information from multiple planes into a single hash value. In the last lecture, we saw how by the sign of the dot product between the normal vector of a plane and a vector representing the data, we could get a relative position relative to that plane. Now, we are going to see how to use this information for multiple planes to get a hash value for the data in the vector space. In order to divide the vector space into manageable regions, we want to use more than one plane.

<img src="images/planes.svg" width="50%"/>

For each plane, we find out whether a vector is on the positive or negative side of that plane. So we get multiple signals, one for each plane and we want to find a way to combine all of those signals into a single **hash value**. This **hash value** will define a particular region within the vector space. So for a single vector, let's say that it is dot product with plane `1` is `3`, so the sign is positive, and the hash value is set to `1` to indicate that the sign is positive. For the second plane, the dot product is `5`, so the sign is again positive and the hash value is `1`. For the third plane, the dot product is `-2`, so the sign is `-1` and the hash value is set to `0` to indicate that the vector `v` is on the negative side of plane `3`. 

$$
\color{red}{P_1v^T = 3,\ \ \ \text{sign}_1 = +1,\ \ \ h_1 = 1} \\
\color{blue}{P_2v^T = 5,\ \ \ \text{sign}_2 = +1,\ \ \ h_2 = 1} \\
\color{orange}{P_3v^T = -2,\ \ \ \text{sign}_3 = -1,\ \ \ h_3 = 0}
$$

To combine these intermediates hash values into a single hash value, we do:

$$
\text{hash} = 2^0 \times h_1 + 2^1 \times h_2 + 2^2 \times h_3 \\
\text{hash} = 1 \times 1 + 2 \times 1 + 4 \times 0 = 3
$$

So, we have multiple planes and it helps us to divide the vector space into smaller sub regions. However, we want to have a single hash value, so we have know which bucket to assign the vectoring. We do this by combining the signals from all the planes into a single hash value. Here are the rules we applied written out, if the sign of the dot product is greater than or equal to `0`, assign the intermediate has value of `1`. Otherwise if the dot product is less than `0`, assign the intermediate hash value of `0`. 

$$
\text{sign}_i \ge 0,\ \ \ \rightarrow h_i = 1 \\
\text{sign}_i < 0,\ \ \ \rightarrow h_i = 0
$$

To combine the intermediate as values, use this formula, this is how we get locality sensitive hashing. 

$$
\text{hash} = \sum\limits_{i}^{H} 2^i \times h_i
$$

We can code this function as:

In [10]:
def side_of_plane(P, v):
    return np.dot(np.array(P), np.array(v).T)
    

def hash_multiple_plane(P_1, v):
    hash_value = 0
    for i, P in enumerate(P_1):
        sign = side_of_plane(P, v)
        hash_i = 1 if sign >= 0 else 0
        hash_value += 2**i * hash_i
    return hash_value

P_1 = [[1, 1], [1, -2], [-2, -2]]
v = [2, 1]
hash_val = hash_multiple_plane(P_1, v)
print('Hash value: ', hash_val)

Hash value:  3


## Approximate Nearest Neighbors

Using locality-sensitive hashing we can make an algorithm that computes k-nearest neighbors much faster than brute search. So far, we have seen that a few planes, such as the three below on the left, can divide the vector space into regions. However, are these planes the best way to divide up the vector space? What if, instead, we divided the vector space like the image on the right? 

<img src="images/multiple_planes.svg" width="50%"/>

In fact, we cannot know for sure which sets of planes is the best way to divide up the vector space, so why not create multiple sets of random planes so that we can divide up the vector space into multiple, independent sets of hash tables. We can think of it like creating multiple copies of the universe, or a multiverse. We can make use of all these different sets of random planes in order to help us find a good set of friendly neighborhood vectors, *i.e*, a set of k-nearest neighbors. 

For example, let's say we have a vector space, and the magenta dot in the middle of the image below represents the transformation of an English word into a French word vector. We are trying to find other French word vectors that may be similar. So maybe one universe of random planes helped us to determine that this magenta vector and the green vectors are all assigned to the same hash bucket. Another entirely different set of random planes helped us determine that these blue vectors are in the same hash bucket as the red vector. A third set of random planes helped us determine that these orange vectors are in the same hash bucket as the magenta vector. 

<img src="images/nearest_neighbors_space.svg" width="35%"/>

By using multiple sets of random planes for locality-sensitive hashing, we have a more robust way of searching the vector space for a set of vectors that are possible candidates to be nearest neighbors. This is called **Approximate Nearest Neighbors** because we are not searching the entire vector space, but just a subset of it. So it is not the absolute k-nearest neighbors, but it is approximately the k-nearest neighbors. We sacrifice some precision in order to gain efficiency in the search. 


Now, let's see how to make a set of random planes in code assuming that word vectors have two dimensions and we want to generate three random planes. We use normal distribution to generate a matrix of three rows and two columns. We create a vector $v$, and for each random plane, and find out whether the vector $v$ is on the positive or negative side of each of these three planes. Notice that instead of using a for loop to work on one plane at a time, we can use `numpy.dot` to do this in one step. The result is that vector $v$ is on the positive side of each of the three random planes. 

In [17]:
num_dimensions = 2
num_planes = 3

random_planes_matrix = np.random.normal(size=(num_planes, num_dimensions))
print("Random planes matrix:")
print(random_planes_matrix)
print()

v = np.array([[2,2]])

def side_of_plane_matrix(P, v):
    dotproduct = np.dot(P, v.T)
    sign_of_dot_product = np.sign(dotproduct)
    return sign_of_dot_product

num_planes_matrix = side_of_plane_matrix(random_planes_matrix, v)
print('Side of the planes')
print(num_planes_matrix)

Random planes matrix:
[[ 1.0728702  -1.01611621]
 [-0.11760075  1.81161565]
 [-0.44825847  0.791393  ]]

Side of the planes
[[1.]
 [1.]
 [1.]]


## Searching documents

Now, we see how we can use fast k-nearest neighbor to search for pieces of text related to a query in a large collection of documents. We simply create vectors for both and find the nearest neighbors. To get ready to perform document search, first, think about how to represent documents as vectors instead of just words as vectors. Let's say we have a documents composed of three words "I love learning". 

$$
\text{I} = [1, 0, 1] \\
\text{love} = [-1, 0, 1] \\
\text{learning} = [1, 0, 1]
$$

We can represent this documents as a vector by finding the word vectors for each individual word and then add them together, so the sum of all these words vectors becomes a documents vector. This document vector has the same dimension as the word vectors. We can then apply document search by using k-nearest neighbors. 

$$
\text{I love learning} = [1, 0, 1] + [-1, 0, 1] + [1, 0, 1] = [1, 0, 3]
$$

Let's see this in code. First, create a mini dictionary for word embeddings and the list of what's contained in the document. We initialize the documents embedding as an array of zeros. Now for each word in a document, we get the word vector if the word exists in the dictionary else zero. We add these all up and return the documents embedding. 

In [18]:
word_embedding = {'I': [1, 0, 1],
                  'love': [-1, 0, 1],
                  'learning': [1, 0, 1]}
words_in_document  = ['I', 'love', 'learning']
document_embedding = np.array([0, 0, 0])
for word in words_in_document:
    document_embedding += word_embedding.get(word, 0)
print(document_embedding)

[1 0 3]
